# Multi Category Classification

In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
#hide
from fastbook import *

## Get and extract the data

In [3]:
?URLs.MNIST

Type:        str
String form: https://s3.amazonaws.com/fast-ai-imageclas/mnist_png.tgz
Length:      56
Docstring:  
str(object='') -> str
str(bytes_or_buffer[, encoding[, errors]]) -> str

Create a new string object from the given object. If encoding or
errors is specified, then the object must expose a data buffer
that will be decoded using the given encoding and error handler.
Otherwise, returns the result of object.__str__() (if defined)
or repr(object).
encoding defaults to sys.getdefaultencoding().
errors defaults to 'strict'.


In [4]:
?untar_data

Signature:
untar_data(
    url: 'str',
    archive: 'Path' = None,
    data: 'Path' = None,
    c_key: 'str' = 'data',
    force_download: 'bool' = False,
    base: 'str' = '~/.fastai',
) -> 'Path'
Docstring: Download `url` using `FastDownload.get`
File:      /anaconda/envs/fastai2/lib/python3.7/site-packages/fastai/data/external.py
Type:      function


In [5]:
path = untar_data(URLs.MNIST)

In [6]:
!ls -l ~/.fastai/data/

total 8
drwxr-x--- 4 fastuser fastuser 4096 Dec 11  2015 mnist_png
drwxr-x--- 4 fastuser fastuser 4096 Sep 26  2018 mnist_sample


In [7]:
(path/'training').ls()

(#10) [Path('/home/fastuser/.fastai/data/mnist_png/training/3'),Path('/home/fastuser/.fastai/data/mnist_png/training/7'),Path('/home/fastuser/.fastai/data/mnist_png/training/9'),Path('/home/fastuser/.fastai/data/mnist_png/training/1'),Path('/home/fastuser/.fastai/data/mnist_png/training/4'),Path('/home/fastuser/.fastai/data/mnist_png/training/2'),Path('/home/fastuser/.fastai/data/mnist_png/training/5'),Path('/home/fastuser/.fastai/data/mnist_png/training/0'),Path('/home/fastuser/.fastai/data/mnist_png/training/8'),Path('/home/fastuser/.fastai/data/mnist_png/training/6')]

In [8]:
threes = (path/'training'/'3').ls().sorted()
sevens = (path/'training'/'7').ls().sorted()
threes

(#6131) [Path('/home/fastuser/.fastai/data/mnist_png/training/3/10.png'),Path('/home/fastuser/.fastai/data/mnist_png/training/3/10000.png'),Path('/home/fastuser/.fastai/data/mnist_png/training/3/10011.png'),Path('/home/fastuser/.fastai/data/mnist_png/training/3/10031.png'),Path('/home/fastuser/.fastai/data/mnist_png/training/3/10034.png'),Path('/home/fastuser/.fastai/data/mnist_png/training/3/10042.png'),Path('/home/fastuser/.fastai/data/mnist_png/training/3/10052.png'),Path('/home/fastuser/.fastai/data/mnist_png/training/3/1007.png'),Path('/home/fastuser/.fastai/data/mnist_png/training/3/10074.png'),Path('/home/fastuser/.fastai/data/mnist_png/training/3/10091.png')...]

In [9]:
?range

Init signature: range(self, /, *args, **kwargs)
Docstring:     
range(stop) -> range object
range(start, stop[, step]) -> range object

Return an object that produces a sequence of integers from start (inclusive)
to stop (exclusive) by step.  range(i, j) produces i, i+1, i+2, ..., j-1.
start defaults to 0, and stop is omitted!  range(4) produces 0, 1, 2, 3.
These are exactly the valid indices for a list of 4 elements.
When step is given, it specifies the increment (or decrement).
Type:           type
Subclasses:     


In [10]:
?str

Init signature: str(self, /, *args, **kwargs)
Docstring:     
str(object='') -> str
str(bytes_or_buffer[, encoding[, errors]]) -> str

Create a new string object from the given object. If encoding or
errors is specified, then the object must expose a data buffer
that will be decoded using the given encoding and error handler.
Otherwise, returns the result of object.__str__() (if defined)
or repr(object).
encoding defaults to sys.getdefaultencoding().
errors defaults to 'strict'.
Type:           type
Subclasses:     DeferredConfigString, _rstr, LSString, include, Keys, InputMode, ColorDepth, CompleteStyle, SortKey, FoldedCase, ...


We want all of the digits, not just 3s and 7s...

In [11]:
digit_paths = [ 
    (path/'training'/str(digit)).ls().sorted()
    for digit in range(10)]

In [12]:
digit_paths[0]

(#5923) [Path('/home/fastuser/.fastai/data/mnist_png/training/0/1.png'),Path('/home/fastuser/.fastai/data/mnist_png/training/0/1000.png'),Path('/home/fastuser/.fastai/data/mnist_png/training/0/10005.png'),Path('/home/fastuser/.fastai/data/mnist_png/training/0/10010.png'),Path('/home/fastuser/.fastai/data/mnist_png/training/0/10022.png'),Path('/home/fastuser/.fastai/data/mnist_png/training/0/10025.png'),Path('/home/fastuser/.fastai/data/mnist_png/training/0/10026.png'),Path('/home/fastuser/.fastai/data/mnist_png/training/0/10045.png'),Path('/home/fastuser/.fastai/data/mnist_png/training/0/10069.png'),Path('/home/fastuser/.fastai/data/mnist_png/training/0/10071.png')...]

Need a tensor to hold the actual images

In [13]:
im0_path = digit_paths[0][0]
im0 = Image.open(im0_path)
im0

In [14]:
tensor(im0)[6:30,6:20]

tensor([[  0,   0,   0,   0,   0,   0,   0,  54, 227, 253, 252, 239, 233, 252],
        [  0,   0,   0,   0,   0,  10,  60, 224, 252, 253, 252, 202,  84, 252],
        [  0,   0,   0,   0,   0, 163, 252, 252, 252, 253, 252, 252,  96, 189],
        [  0,   0,   0,   0,  51, 238, 253, 253, 190, 114, 253, 228,  47,  79],
        [  0,   0,   0,  48, 238, 252, 252, 179,  12,  75, 121,  21,   0,   0],
        [  0,   0,  38, 165, 253, 233, 208,  84,   0,   0,   0,   0,   0,   0],
        [  0,   7, 178, 252, 240,  71,  19,  28,   0,   0,   0,   0,   0,   0],
        [  0,  57, 252, 252,  63,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0, 198, 253, 190,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 76, 246, 252, 112,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 85, 252, 230,  25,   0,   0,   0,   0,   0,   0,   0,   0,   7, 135],
        [ 85, 252, 223,   0,   0,   0,   0,   0,   0,   0,   0,   7, 131, 252],
        [ 85, 252, 145,   0,   0,   0,  

Make it look a bit prettier with a pandas DataFrame

### pandas.DataFrame
class pandas.DataFrame(data=None, index=None, columns=None, dtype=None, copy=None)[source]
Two-dimensional, size-mutable, potentially heterogeneous tabular data.

Data structure also contains labeled axes (rows and columns). Arithmetic operations align on both row and column labels. Can be thought of as a dict-like container for Series objects. The primary pandas data structure.

In [15]:
im0_t = tensor(im0)
df = pd.DataFrame(im0_t[6:30,6:20])
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0,0,0,0,0,0,0,54,227,253,252,239,233,252
1,0,0,0,0,0,10,60,224,252,253,252,202,84,252
2,0,0,0,0,0,163,252,252,252,253,252,252,96,189
3,0,0,0,0,51,238,253,253,190,114,253,228,47,79
4,0,0,0,48,238,252,252,179,12,75,121,21,0,0
5,0,0,38,165,253,233,208,84,0,0,0,0,0,0
6,0,7,178,252,240,71,19,28,0,0,0,0,0,0
7,0,57,252,252,63,0,0,0,0,0,0,0,0,0
8,0,198,253,190,0,0,0,0,0,0,0,0,0,0
9,76,246,252,112,0,0,0,0,0,0,0,0,0,0
